In [2]:
from astropy.io import fits
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [93]:
config_path = '/home/hyz/da/testhub/cmr_test/'

检查flg是否包含16的函数

In [94]:
def flg2num(flg):
    flg_list = []
    for i in np.linspace(8,0,9):
        num_i = 2**i
        if flg >= num_i:
            flg = flg%num_i
            flg_list.append(num_i)
    return flg_list

def flgcheck16(flg):
    flg_list = flg2num(flg)
    flg_list = np.array(flg_list)
    if len(flg_list[flg_list==16.]) == 1:
        return 1
    else:
        return 0

def flgncheck16(flg):
    for i in np.linspace(8,5,4,dtype=int):
        num_i = 2**i
        flg %= num_i
    flg //= 16
    return flg

flgncheck16(np.array([[257,234,412,25,156],[1,0,24,36,76]]))

array([[0, 0, 1, 1, 1],
       [0, 0, 1, 0, 0]])

像素数计算函数

In [95]:
def mulapix():
    with open(config_path+"test_file_list.txt") as tfl:
        flist = tfl.readlines()
    nump_flgl, nump_crsl, nump_checkl = [],[],[]
    
    for _file in flist:
        _crsf = _file.split('SCI')[0]+"CRS"+_file.split('SCI')[1].rstrip()+"L0_1.fits"
        # _flgf = _file + 'flg.fits.sisteen'      # flg提取标记16后图
        _flgf = _file + 'flg.fits.allone'             # flg原图
        _checkf = _file + 'checka.fits'
        with fits.open(_crsf) as hdu_crs:
            crs = hdu_crs[1].data
            if os.path.exists(_flgf) is False:
                os.system("cp %s %s" %(_flgf[:-7], _flgf))
                with fits.open(_flgf,mode='update') as hdu_flg:
                    flg = hdu_flg[1].data
                    flg = flg / (flg+0.01)
                    flg = np.int8(np.round(flg))
                    hdu_flg[1].data = flg
                    hdu_flg.flush()
            else:
                with fits.open(_flgf) as hdu_flg:
                    flg = hdu_flg[1].data
            check = crs*flg
            hdu_crs[1].data = check
            hdu_crs.writeto(_checkf, overwrite=True)
        nump_flg = np.sum(np.ones_like(flg)[flg>0])
        nump_check = np.sum(np.ones_like(check)[check>0])
        nump_crs = np.sum(np.ones_like(crs)[crs>0])
        nump_flgl.append(nump_flg)
        nump_crsl.append(nump_crs)
        nump_checkl.append(nump_check)
    pix_table = pd.DataFrame({  'id':_file[-6:-1],
                                'pixel_crs':nump_crsl,
                                'pixel_flg':nump_flgl,
                                'pixel_check':nump_checkl  })
    pix_table["pixel-loss find rate"] = (1-pix_table['pixel_check']/pix_table['pixel_crs'])*100
    pix_table['pixel-false alarm rate'] = (pix_table['pixel_flg']-pix_table['pixel_check'])/pix_table['pixel_flg']*100
    return pix_table

    

sex运行与reg生成函数

In [98]:
def sexaread():

    with open(config_path+"test_file_list.txt") as tfl:
        flist = tfl.readlines()

    sex_crsl, sex_checkl, sex_flgl = [], [], []
    
    sufl = ['crs','check','flg']
    for _file in flist:
        for _suf in sufl:
            _acat = config_path + 'acat/' + _file[-6:] + _suf + 'a.acat'
            if _suf == 'crs':
                file = _file.split('SCI')[0]+"CRS"+_file.split('SCI')[1].rstrip()+"L0_1.fits"
            elif _suf == 'check':
                file = _file.rstrip() + _suf+"a.fits"
            elif _suf == 'flg':
                # file = _file.rstrip() + _suf + '.fits.sisteen'      # flg提取图
                file = _file.rstrip() + _suf + '.fits.allone'              # flg原图
            print(file,"\n",_acat)
            
            os.system("source-extractor -c " + config_path + "cmr.sex " + file    
                    + ' -CATALOG_NAME ' + _acat)
            
            with fits.open(_acat) as hdus:
                cat = Table(hdus[2].data)
                lenc = len(cat)
                _reg = _acat[0:-5] + '.reg'
                with open(_reg, 'w') as f:
                    f.write("# Region file format: DS9 version 4.1\nglobal color=green dashlist=8 3 width=1 font='helvetica 10 normal roman' select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\nimage\n")
                    for i in range(lenc):
                        f.writelines("ellipse("+str(cat['XWIN_IMAGE'][i])+","+str(cat['YWIN_IMAGE'][i])+
                                    ","+str(4*cat['AWIN_IMAGE'][i])+","+str(4*cat['BWIN_IMAGE'][i])+","
                                    +str(cat['THETAWIN_IMAGE'][i])+")\n")
            if _suf == 'crs':
                sex_crsl.append(lenc)
            if _suf == 'check':
                sex_checkl.append(lenc)
            elif _suf == 'flg':
                sex_flgl.append(lenc)
    sex_table = pd.DataFrame({  'id':_file[-6:-1],
                                'source_crs': sex_crsl,
                                'source_flg': sex_flgl,
                                'source_check': sex_checkl  })
    sex_table["source-loss find rate"] = (1-sex_table['source_check']/sex_table['source_crs'])*100
    sex_table['source-false alarm rate'] = (sex_table['source_flg']-sex_table['source_check'])/sex_table['source_flg']*100
    return sex_table


检查erg差异并生成新的erg函数

In [99]:
def regsub(former,latter,new):
    with open(former,'r') as erg1:
        with open(latter,'r') as erg2:
            with open(new,'w') as erg_n:
                reg_list = erg1.readlines()
                reg_listc = reg_list.copy()
                reg1_list = erg2.readlines()
                for line in reg_list:
                    r1 = line.split("(",1)
                    if len(r1) > 1:
                        r1 = r1[1]
                        r1 = r1.split(",",4)
                        r1x = float(r1[0])
                        r1y = float(r1[1])
                        for line1 in reg1_list:
                            r2 = line1.split("(",1)
                            if len(r2) > 1:
                                r2 = r2[1]
                                r2 = r2.split(",",4)
                                r2x = float(r2[0])
                                r2y = float(r2[1])
                                if abs(r1x-r2x)+abs(r1y-r2y) < 1:
                                    reg_listc.remove(line)
                                    reg1_list.remove(line1)
                                    break
                                
                erg_n.writelines("# Region file format: DS9 version 4.1\nglobal color=yellow dashlist=8 3 width=1 font='helvetica 10 normal roman' select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\nimage\n")
                erg_n.writelines(reg1_list[3:])
                erg_n.writelines("# Region file format: DS9 version 4.1\nglobal color=magenta dashlist=8 3 width=1 font='helvetica 10 normal roman' select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\nimage\n")
                erg_n.writelines(reg_listc[3:])
                erg_n.writelines("text 4500 8900 {yellow: later-former} # color=white \ntext 4500 8800 {magenta: former-later} # color=white")
                num_con1 = len(reg_listc)-3
                num_con2 = len(reg1_list)-3
                return [num_con1,num_con2]

def regdiff():
    with open(config_path+"test_file_list.txt") as tfl:
        flist = tfl.readlines()
    num_ccl,num_fcl,num_cfl = [],[],[]
    for _file in flist:
        _pre = _file.split("CSST",1)[0] + "/acat/" + _file[-6:]
        _crsr = _pre+"crsa.reg"
        _flgr = _pre+"flga.reg"
        _checkr = _pre+"checka.reg"
        _cc = _pre+"crsa-checka.reg"
        _fc = _pre+"flga-checka.reg"
        _cf = _pre+"crsa-flga.reg"
        num_ccl.append(regsub(_crsr,_checkr,_cc))
        num_fcl.append(regsub(_flgr,_checkr,_fc))
        num_cfl.append(regsub(_crsr,_flgr,_cf))
    num_ccl = np.array(num_ccl)
    num_fcl = np.array(num_fcl)
    num_cfl = np.array(num_cfl)
    rd_table = pd.DataFrame({   'crs-check':num_ccl[:,0],
                                'check-crs':num_ccl[:,1],
                                'flg-check':num_fcl[:,0],
                                'check-flg':num_fcl[:,1],
                                'crs-flg':num_cfl[:,0],
                                'flg-crs':num_cfl[:,1]})
    return rd_table



In [100]:
pix_table = mulapix()
sex_table = sexaread()
a_table = pd.concat([pix_table,sex_table],axis=1)
a_table = a_table.T.drop_duplicates().T


/home/hyz/da/testhub/cmr_test/CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_12_L0_1.fits 
 /home/hyz/da/testhub/cmr_test/acat/25_12_crsa.acat



> WARNING: NTHREADS != 1 ignored: this build of Source Extractor is single-threaded

> 
----- Source Extractor 2.25.0 started on 2022-11-22 at 12:29:29 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_12_L0_1.fits
----- Measuring from: CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_12_L0_1.fits [1/1]
      "Unnamed" / no ext. header / 9216x9232 / 16 bits (integers)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:  256
> Setting up background map at line:  512
> Setting up background map at line:  768
> Setting up background map at line: 1024
> Setting up background map at line: 1280
> Setting up background map at line: 1536
> Setting up background map at line: 1792
> Setting up background map at line: 2048
> Setting up background map at line: 2304
> Setting up background map at line: 2560
> Setting up background map a

/home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_checka.fits 
 /home/hyz/da/testhub/cmr_test/acat/25_12_checka.acat



> WARNING: NTHREADS != 1 ignored: this build of Source Extractor is single-threaded

> 
----- Source Extractor 2.25.0 started on 2022-11-22 at 12:29:35 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_checka.fits
----- Measuring from: CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_checka.fits [1/1]
      "Unnamed" / no ext. header / 9216x9232 / 32 bits (integers)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:  256
> Setting up background map at line:  512
> Setting up background map at line:  768
> Setting up background map at line: 1024
> Setting up background map at line: 1280
> Setting up background map at line: 1536
> Setting up background map at line: 1792
> Setting up background map at line: 2048
> Setting up background map at line: 2304
> Setting up background map at line: 2560
> Setting up background m

/home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_flg.fits.allone 
 /home/hyz/da/testhub/cmr_test/acat/25_12_flga.acat


> Setting up background map at line:  256
> Setting up background map at line:  512
> Setting up background map at line:  768
> Setting up background map at line: 1024
> Setting up background map at line: 1280
> Setting up background map at line: 1536
> Setting up background map at line: 1792
> Setting up background map at line: 2048
> Setting up background map at line: 2304
> Setting up background map at line: 2560
> Setting up background map at line: 2816
> Setting up background map at line: 3072
> Setting up background map at line: 3328
> Setting up background map at line: 3584
> Setting up background map at line: 3840
> Setting up background map at line: 4096
> Setting up background map at line: 4352
> Setting up background map at line: 4608
> Setting up background map at line: 4864
> Setting up background map at line: 5120
> Setting up background map at line: 5376
> Setting up background map at line: 5632
> Setting up background map at line: 5888
> Setting up background map at lin

In [101]:
rd_table = regdiff()
c_table = pd.concat([a_table,rd_table],axis=1)
c_table['diff-loss find rate'] = (c_table['crs-check']-c_table['check-crs'])/c_table['source_crs']*100
c_table['diff-false alarm rate'] = (c_table['flg-check']-c_table['check-flg'])/c_table['source_check']*100
c_table


,id,pixel_crs,pixel_flg,pixel_check,pixel-loss find rate,pixel-false alarm rate,source_crs,source_flg,source_check,source-loss find rate,source-false alarm rate,crs-check,check-crs,flg-check,check-flg,crs-flg,flg-crs,diff-loss find rate,diff-false alarm rate
0,25_12,269980,236350,217692,19.367361,7.894225,11166,10801,10855,2.785241,-0.499954,444,133,376,430,790,425,2.785241,-0.497467


In [102]:
c_table.T

,0
id,25_12
pixel_crs,269980
pixel_flg,236350
pixel_check,217692
pixel-loss find rate,19.367361
pixel-false alarm rate,7.894225
source_crs,11166
source_flg,10801
source_check,10855
source-loss find rate,2.785241


使用提取前和提取后的check差异

In [103]:
regsub('./acat/25_12_check.reg','./acat/25_12_checka.reg','./acat/25_12_check-checka.reg')

[1, 3]

查看检测结果

* flg-check
* cmr-check
* cmr-flg
* cmr-flga
* check-checka

In [104]:
stop

NameError: name 'stop' is not defined

In [ ]:
os.system("ds9 /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_flg.fits.sisteen \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_flg-check.reg \
/home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_check.fits \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_flg-check.reg")

0

In [ ]:
os.system("ds9 /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_12_L0_1.fits \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_crs-check.reg \
/home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_check.fits \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_crs-check.reg")

0

In [ ]:
os.system("ds9 /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_12_L0_1.fits \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_crs-flg.reg \
/home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_flg.fits.sisteen \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_crs-flg.reg")

In [ ]:
os.system("ds9 /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_12_L0_1.fits \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_crs-flg.reg \
/home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_flg.fits \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_crs-flg.reg")

In [ ]:
os.system("ds9 /home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_check.fits \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_check-checka.reg \
/home/hyz/da/testhub/cmr_test/CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_checka.fits \
-region /home/hyz/da/testhub/cmr_test/acat/25_12_check-checka.reg")

读取acat并生成erg和记录表

In [ ]:
file = '/home/hyz/da/testhub/cmr_test///CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_flg.fits'
file = file.split('SCI')[0]+"CRS"+file.split('SCI')[1].rstrip()[:-8]+"L0_1.fits"
os.system("ds9 %s"% file)

0

In [ ]:
'flg.fits.sisteen'[:-8]

'flg.fits'

* 测试

In [ ]:
with fits.open("CSST_MSC_MS_SCI_20241212080710_20241212080940_100000025_12_flg.fits") as hdu_flg:
    flg = hdu_flg[1].data
np.sum(flg)

3585092

In [ ]:
for i in range(18):
    print('CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_%s_L0_1.fits' %(i+6))

CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_6_L0_1.fits
CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_7_L0_1.fits
CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_8_L0_1.fits
CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_9_L0_1.fits
CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_10_L0_1.fits
CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_11_L0_1.fits
CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_12_L0_1.fits
CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_13_L0_1.fits
CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_14_L0_1.fits
CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_15_L0_1.fits
CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_16_L0_1.fits
CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_17_L0_1.fits
CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_18_L0_1.fits
CSST_MSC_MS_CRS_20241212080710_20241212080940_100000025_19_L0_1.fits
CSST_MSC_MS_CRS_20241212080710_2024121

In [ ]:
"CHECK" in "cmr_test/CSST_CRSCHECK_25_12.fits"

True